In [ ]:
import os
import geopandas as gpd
gpd.options.io_engine = 'pyogrio'

In [ ]:
awi_attribution_html = 'Natural England, NatureScot, Scottish Forestry, Natural Resources Wales. Contains Ordnance Survey Data. © <a href="https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/">Crown copyright and database right 2006-2022</a>'
nfi_attribution_html = 'Forestry Commission. Contains Ordnance Survey Data. © <a href="https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/">Crown copyright and database right 2012-2022</a>'
combined_attribution_html = 'Forestry Commission, Natural England, NatureScot, Scottish Forestry, Natural Resources Wales. Contains Ordnance Survey Data. © <a href="https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/">Crown copyright and database right 2006-2022</a>'

In [ ]:
year = 2022
nfi_awi_overlay = gpd.read_parquet(f'../data/processed/gb_nfi_awi_overlay_{year}.parquet')
nfi_awi_overlay.geometry = nfi_awi_overlay.geometry.to_crs(epsg=4326)
nfi_awi_overlay.geometry = nfi_awi_overlay.geometry.simplify(1e-5)
nfi_awi_overlay.to_file(f'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson', driver='GeoJSONSeq')

In [ ]:
# PASS: -zg
# Produces a stable result but has artifacting around zoom levels 5-6, 7-9 (linear, cubic)
# 2m 17s for geojsonseq | 2m 40s w/o -P
!tippecanoe -P -zg -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# PASS: -zg --gamma=2
# Time: 5m, Size: 260MB
# Works not bad I must say, generally happy with that!
!tippecanoe -P -zg --gamma=1 -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# PASS: -zg --gamma=1
# Time: 4m 15s, Size: 265MB
# The artifacts from no gamma are definitely present but are far less noticeable. Good enough for now.
# BUG? The tippecanoe's file name lookup seems to be case-insensitive, so it's overwriting the previous file if they're just using z9 and Z9 (casing) to differentiate.
!tippecanoe -P -zg -z4 -o 'tiles/experimental/gb_nfi_awi_overlay_{year}-p1.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f
!tippecanoe -P -zg -Z4 -z6 --drop-smallest-as-needed -o 'tiles/experimental/gb_nfi_awi_overlay_{year}-p2.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f
!tippecanoe -P -zg -Z6 -z9 --gamma=2 -o 'tiles/experimental/gb_nfi_awi_overlay_{year}-p3.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f
!tippecanoe -P -zg -Z9 -o 'tiles/experimental/gb_nfi_awi_overlay_{year}-p4.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

!tile-join -pk --force -o 'tiles/experimental/gb_nfi_awi_overlay_{year}-p12.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}-p1.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}-p2.mbtiles'
!tile-join -pk --force -o 'tiles/experimental/gb_nfi_awi_overlay_{year}-p34.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}-p3.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}-p4.mbtiles'
!tile-join -pk --force -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}-p12.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}-p34.mbtiles'

In [ ]:
# PASS: --drop-smallest-as-needed
## The clashing is gone, but an enormous amount of features is not represented, in a way that's not satisfying. They do start to appear as the zoom goes and it's the smaller ones that are dropped but I think it can still be done better.
# Time: 5m, Size: forgot to check :S
!tippecanoe -P -zg --drop-smallest-as-needed -o 'tiles/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# PASS: --hilbert --coalesce --no-duplication --detect-shared-borders
## The clashing is gone, but an enormous amount of features is not represented, in a way that's not satisfying. They do start to appear as the zoom goes and it's the smaller ones that are dropped but I think it can still be done better.
# Time: 5m, Size: 275MB
# This gives great results zoom levels 8 and above. Still has a pronounced line in England for zoom 5-6
!tippecanoe -P -zg --hilbert --coalesce --no-duplication --detect-shared-borders -o 'tiles/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# PASS: --hilbert --coalesce --no-duplication --detect-shared-borders --drop-smallest-as-needed
## The clashing is gone, but an enormous amount of features is not represented, in a way that's not satisfying. They do start to appear as the zoom goes and it's the smaller ones that are dropped but I think it can still be done better.
# Time: 6m 45s, Size: 245MB
# Mediocre result, a lot of features dropped, bad
!tippecanoe -P -zg --hilbert --coalesce --no-duplication --detect-shared-borders --drop-smallest-as-needed -o 'tiles/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# PASS: --hilbert --coalesce --no-duplication --detect-shared-borders --drop-fraction-as-needed
# Time: 5m, Size: 238MB
# Not better than the usual --drop-fraction-as-needed
!tippecanoe -P -zg --hilbert --coalesce --no-duplication --detect-shared-borders --drop-fraction-as-needed -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# PASS: -zg --hilbert --coalesce-smallest-as-needed --detect-shared-borders --gamma=1
# Time: x, Size: x
# Mostly fine at low levels, but drops large polygons at high zoom levels, unacceptable
!tippecanoe -P -zg --hilbert --coalesce --detect-shared-borders --gamma=1 -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# FAIL: --coalesce-fraction-as-needed
# Time: x, Size: x
# Hangs on 'Going to try keeping 0.00% of the features to make it fit 46.1%  5/15/10 '
!tippecanoe -P -zg --coalesce-fraction-as-needed -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# FAIL: --coalesce-densest-as-needed
# Time: x, Size: x
# Hangs on 'Going to try keeping 0.00% of the features to make it fit 46.1%  5/15/10 '
!tippecanoe -P -zg --coalesce-densest-as-needed -o 'tiles/experimental/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/experimental/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

In [ ]:
# FAIL: --coalesce-smallest-as-needed
## Crashes at 'Going to try keeping the biggest 0.00% of the features to make it fit tile 5/15/10 size is 717848 with detail 12, >500000'
!tippecanoe -P -zg --coalesce-smallest-as-needed -o 'tiles/gb_nfi_awi_overlay_{year}.mbtiles' 'tiles/gb_nfi_awi_overlay_{year}.geojson' -n 'gb_nfi_awi_overlay_{year}' -l 'gb_nfi_awi_overlay_{year}' --attribution='{combined_attribution_html}' --description='National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain' -f

### Things that don't work

`--no-tiny-polygon-reduction` doesn't work because it vastly increases the number of features on lower zoom levels.
`--no-duplication` "cuts out" the features from polygons near

### To Try:
`--detect-shared-borders`: In the manner of TopoJSON, detect borders that are shared between multiple polygons and simplify them identically in each polygon. This takes more time and memory than considering each polygon individually.
- regular setting <5 level, then 5-9 try something else, then again regular setting 9 onward. Maybe simplify on lower zoom levels?
`--coalesce`: Coalesce consecutive features that have the same attributes. This can be useful if you have lots of small polygons with identical attributes and you would like to merge them together.
`--hilbert`: Put features in Hilbert Curve order instead of the usual Z-Order. This improves the odds that spatially adjacent features will be sequentially adjacent, and should improve density calculations and spatial coalescing. It should be the default eventually.